In [ ]:
from funcs.CSIFuncs import *
from funcs.HelperFuncs import *

from multiprocessing import Pool
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import itertools

%matplotlib inline

pd.options.display.float_format = '{:6f}'.format

import warnings
warnings.filterwarnings("ignore")


## Setting configs


In [ ]:
# Directory storing the CSI and WLAN captures
resources_dir = "/Volumes/tim_details/tim_honours/CAPTURES"

# Directory to save plots to
plt_dir = "/Users/timothylee/Desktop/Uni/Yr5/Honours/honours_thesis/figures/plt_figs/"

# Supress pd scientific notation
pd.set_option('display.float_format', '{:.6f}'.format)

# Resolution of plots
plt.rcParams["figure.dpi"] = 100 # 300
plt.rcParams["figure.dpi"] = 500 # 300

# Backend to generate plots
# mpl.use("agg")
# %matplotlib ipympl
%matplotlib inline

# plt figure style
fig_style = "seaborn-v0_8-whitegrid"

# colormaps
cmap_qual = "pastel"
cmap_seq = "viridis"
cmap_cycl = "twilight"


## Storing CSI data as h5 tables

In [ ]:
procs = 4

dirs = glob("/Volumes/tim_details/tim_honours/CAPTURES/*/*")

for dir in dirs:
    # Setting up dirs
    clean_dir_junk(os.path.join(dir, "csi"))
    make_dir(os.path.join(dir, "csi_h5"))
    # Running multiprocessing: csi pcap to h5
    starargs = [(dir, get_name(i)) for i in os.listdir(os.path.join(dir, "csi"))]
    with Pool(procs, initializer=csi_to_df_init_mp) as pool:
        pool.starmap(csi_to_df_mp, starargs)

## Reading and Processing Nexmon CSI PCAP files


In [ ]:
# Reading csi info (including metadata)
frames = read_csi(csi_fp)
# Checking that the CSI is valid before continuing
if check_csi(frames["csi"]):
    # Processing csi info
    csi = process_csi(frames["csi"], True, True, 5)
    # Making csi amplitude plots
    # fig, axes = CSIFuncs.plot_all(csi)
    # fig.savefig(plot_fp)
    # plt.close(fig)

### Visual: example CSI comparison of two devices

In [ ]:
# Two Pi examples and two PC examples
fps = [
    [
        "/Volumes/tim_details/tim_honours/CAPTURES/client_pi_200/v=A3gUpodXMv0/csi_h5/cap_5.h5",
        "/Volumes/tim_details/tim_honours/CAPTURES/client_pi_200/v=A3gUpodXMv0/csi_h5/cap_7.h5",
    ],
        [
        "/Volumes/tim_details/tim_honours/CAPTURES/client_pc_200/v=A3gUpodXMv0/csi_h5/cap_2.h5",
        "/Volumes/tim_details/tim_honours/CAPTURES/client_pc_200/v=A3gUpodXMv0/csi_h5/cap_5.h5",
    ]
]
dev_map = {0: "RPi4", 1: "PC"}

vmin = 0
vmax = 1000

# PLOTTING
with plt.style.context(fig_style):
    fig = plt.figure(figsize=(9, 9), layout="constrained")
    axes = fig.subplot_mosaic(
        """
        ab
        cd
        """
    )

for i, fps_i in enumerate(fps):
    for j, fp in enumerate(fps_i):
        # Reading CSI data from h5 file
        csi_df = pd.read_hdf(fp, key=H5_CSI_KEY, mode="r")
        # Getting and processing CSI matrix
        csi = df_to_csi_matrix(csi_df)
        csi = process_csi(csi, True, True, 5)
        # Getting the amplitudes of the CSI complex values
        csi = np.abs(csi)
        # Making heatmap
        ax_id = chr(ord('a')+i*2+j)
        im = axes[ax_id].pcolormesh(
            make_subc_matrix(*csi.shape)[0],
            np.arange(csi.shape[0]),
            csi,
            cmap=cmap_seq,
            norm=Normalize(
                vmin=vmin,
                vmax=vmax,
            ),
        )
        # CUSTOMISING APPEARANCE
        # Seting ylim at 6000 frames for each statistic/measure (each column)
        axes[ax_id].set_ylim(0, 6000)
        # Remove axes titles
        axes[ax_id].set_xlabel(None)
        axes[ax_id].set_ylabel(None)
        # Adding subplot titles
        axes[ax_id].set_title(f"{ax_id})", loc='left', fontsize='medium')
        axes[ax_id].set_title(f"{dev_map[i]} - Sample {j+1}")

# Make overall colourbar for all axes
fig.colorbar(im, ax=list(axes.values()))

# Adding axes titles for the left and bottom edges
# Top row titles - label group
for i in ["c", "d"]:
    axes[i].set_xlabel("Subcarrier Index")
# Left column y-axis - statistic/measure:
for i in ["a", "c"]:
    axes[i].set_ylabel("Frame Number")
# Setting overall figure title
fig.suptitle("CSI Amplitudes for different\nDevices", fontsize="xx-large")

# Plot fig
fig.savefig(os.path.join(plt_dir, "CSI_intro_example.png"))
# fig.clf()
# plt.close()

## Analysing CSI time-series characteristics (packets per second)

In [ ]:
# Choosing labels and instances
devices = {
    "client_pc_200": "PC",
    "client_pi_200": "RPi4",
}
videos = {
    "v=A3gUpodXMv0": "Vid 1",
    "v=gxxqdrrpgZc": "Vid 2",
    # "v=mkWKZWMokdI": "Vid 3",
    # "v=NSW5u1RTxEA": "Vid 4",
}
instances = [
    "cap_1",
    "cap_10",
    "cap_50",
    # "cap_70",
    # "cap_80",
]
labels = pd.DataFrame(columns=["devices", "videos", "instances"])
for i in itertools.product(devices, videos, instances):
    labels.loc[len(labels)] = i


# Agg: frame counts, mean amplitude of each subc
agg_dict = {"frame.time_relative": "count"}
for i in np.arange(64):
    agg_dict[f"csi_{i}"] = lambda x: np.nanmean(np.abs(x))

# Making combined binned DF of all labels and instances
interval = 1.0
df_binned_comb = pd.DataFrame()
for i in labels.index:
    # Getting the particular label details
    row = labels.loc[i]
    # Reading the csi h5 file
    df = pd.read_hdf(
        os.path.join(resources_dir, row["devices"], row["videos"], "csi_h5", f"{row['instances']}.h5"),
        key=H5_CSI_KEY,
        mode="r",
    )
    # Converting df CSI columns from "r" and "i" columns to combined complex
    csi = df_to_csi_matrix(df)
    df = df.loc[:,["csi_" not in i for i in df.columns]]
    for i in np.arange(csi.shape[1]):
        df[f"csi_{i}"] = csi[:, i]

    # Making the binned df
    df_binned = ts_bin_df(df, interval, agg_dict)
    # Imputing missing values
    df_binned = df_binned.fillna(0)
    # df_binned = df_binned.interpolate(method="linear", axis=0)
    # Making cumulative frame counts
    df_binned["Cumulative Frames"] = df_binned["frame.time_relative"].cumsum()

    # Adding label combo to group instances
    df_binned["labels"] = f"{row['devices']} - {row['videos']}"
    df_binned["instances"] = row["instances"]
    # Setting the ts_bins index as a column
    df_binned["ts_bins"] = df_binned.index

    # Concatenating the instance's binned df to the overall binned df
    df_binned_comb = pd.concat([df_binned_comb, df_binned], axis=0, ignore_index=True)

# Renaming columns
df_binned_comb = df_binned_comb.rename(
    columns={
        "frame.time_relative": "Frames",
        "Cumulative Frames": "Cumulative Frames",
    }
)

# PLOTTING
with plt.style.context(fig_style):
    fig = plt.figure(figsize=(12, 12), layout="constrained")
    axes = fig.subplots(
        nrows=len(list(itertools.product(devices, videos))),  # classes
        ncols=1,  # features
    )

# Cumulative packets across time bins
for i, lab in enumerate(df_binned_comb["labels"].unique()):
    # Filtering df for the given label
    df_1 = df_binned_comb[df_binned_comb["labels"] == lab]
    sns.lineplot(
        data=df_1,
        x="ts_bins",
        y="Frames",
        hue="instances",
        alpha=0.8,
        ax=axes[i],
    )

# CSI amplitudes across time (averaged within time bins)
# csi_cols = [f"csi_{i}" for i in np.arange(csi.shape[1])]
# axes[1].pcolormesh(
#     df_binned[csi_cols].T,
# )


## Getting summaries of CSI data

In [ ]:
# GETTING SUMMARIES OF A DIRECTORY OF CAPTURES:
# nfiles, nframes, statistics
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import CSIFuncs

dirs = glob("/Volumes/tim_details/tim_honours/CAPTURES/*/*")
dirs = ["/Volumes/tim_details/tim_honours/CAPTURES/client_pi_200/v=mkWKZWMokdI"]

# Getting a summary of each different CSI experiment setup
res_arr = {}
for dir in dirs:
    print(dir)
    fps = glob(os.path.join(dir, "csi/*"))
    res_arr[dir] = CSIFuncs.get_summaries(fps)

fig, ax = plt.subplots()
for i in res_arr:
    res = res_arr[i]
    ax.hist(
        res["nframes"],
        alpha=0.2,
        label=f"{i}",
    )
# plt.legend()

## Combining CSI from each label combination into an array of 2D arrays
- 0 axis: The individual csi capture matrix (i.e. the file)
- 1 axis: The frames (note: must be trimmed or padded - same size across all)
- 2 axis: The subcarrier


In [ ]:
import pandas as pd
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

dirs = glob("/Volumes/tim_details/tim_honours/CAPTURES/*/*")
nframes = 10000

for dir in dirs:
    print(dir)
    fps = glob(os.path.join(dir, "csi/*"))
    comb_csi = CSIFuncs.combine_csis(fps, nframes)
    np.save(os.path.join(dir, "csi_all.npy"), comb_csi)

# fig, ax = plt.subplots()
# CSIFuncs.hist2d_subc(np.abs(csi), fig, ax)
# fig, ax = plt.subplots()
# CSIFuncs.hist2d_complex(csi, fig, ax)
# CSIFuncs.plot_all(csi)